In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Sequential

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K


In [4]:
!pip install livelossplot


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
from tensorflow import keras
from tensorflow.keras.callbacks import CSVLogger
from livelossplot import PlotLossesKeras as PlotLossesCallback

In [4]:
# dimensions of our images.
img_width, img_height = 224, 224

In [5]:
train_data_dir = 'cats-and-dogs/train'
validation_data_dir = 'cats-and-dogs/test'
nb_train_samples = 40
nb_validation_samples = 10
epochs = 2
batch_size = 4

In [6]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

NameError: name 'K' is not defined

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape,activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [11]:
model.compile(loss='binary_crossentropy', # or categorical_crossentropy
              optimizer='rmsprop',# or adagrad
              metrics=['accuracy'])

In [12]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [13]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [14]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

print(train_generator.class_indices)

Found 40 images belonging to 2 classes.
{'cat': 0, 'dog': 1}


In [15]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 10 images belonging to 2 classes.


In [16]:
import scipy.ndimage
history=model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples ,
    epochs=2,
    validation_data=validation_generator,
    validation_steps=len(validation_generator.filenames)// batch_size,
   # callbacks=[PlotLossesKeras,CSVLogger('test.csv',append=False),';'],verbose=1)


NameError: name 'PlotLossesKeras' is not defined

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
acc=history.history['accuracy']


In [ ]:
val_acc=history.history['val_accuracy']

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(len(acc))
plt.plot(epochs,acc,label='Training Accuracy')
plt.plot(epochs,val_acc,label="Validation accuracy")
plt.title('Training and validation accuracy')
plt.legend(loc='upper left')
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.figure()

In [77]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report
ypred=model.predict_generator(validation_generator,10//
                             batch_size+1)
x=ypred
ypred=np.argmax(ypred,axis=1)
conf=confusion_matrix(validation_generator.classes,ypred)
print(conf)

[[5 0]
 [5 0]]


In [60]:
np.argmax?

In [80]:
print(x)
#ypred=np.argmax(ypred,axis=1)
#ypred[0]=7
x.shape
#x[7]=100
x=np.argmax(x,axis=0)
x

[0 0 0 0 0 0 0 0 0 0]


0

In [51]:
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix
y_pred=model.predict_generator(validation_generator,1)
x=y_pred
y_pred=np.argmax(y_pred,axis=1)
print('Confusion matrix')
print(confusion_matrix(validation_generator.classes,y_pred))
print('Classification report')
target_names=['Cats','Dogs']
print(classification_report(validation_generator.classes,y_pred,target_names=target_names))

Confusion matrix


ValueError: Found input variables with inconsistent numbers of samples: [10, 4]

In [52]:
import numpy as np
#np.argmax?
y_pred.shape
x.shape

(4, 1)

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from os import listdir
from os.path import isfile, join

In [19]:
# dimensions of our images
img_width, img_height = 224, 224

In [ ]:
# load the model we saved
#model = load_model('model.h5')
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
mypath = "cats-and-dogs/test/image/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
# predicting images
dog_counter = 0 
cat_counter  = 0
for file in onlyfiles:
    img = image.load_img(mypath+file, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict_classes(images, batch_size=10)
    classes = classes[0][0]
    
    if classes == 0:
        print(file + ": " + 'cat')
        cat_counter += 1
    else:
        print(file + ": " + 'dog')
        dog_counter += 1
print("Total Dogs :",dog_counter)
print("Total Cats :",cat_counter)